In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold 
from mordred import Calculator, descriptors
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import Descriptors
from rdkit.Chem.EState import Fingerprinter
from rdkit.Chem import PandasTools
from sklearn.preprocessing import StandardScaler 
from sklearn.svm import SVR
datatra=pd.read_csv('BMF_train.csv',encoding = 'gbk')
dataext=pd.read_csv('BMF_test.csv',encoding = 'gbk')
y=datatra.iloc[:,1]
y2=dataext.iloc[:,1]
PandasTools.AddMoleculeColumnToFrame(datatra,'SMILES','Molecule')
PandasTools.AddMoleculeColumnToFrame(dataext,'SMILES','Molecule')
calc = Calculator(descriptors, ignore_3D=True)
X= pd.DataFrame(calc.pandas(datatra['Molecule']))
X2= pd.DataFrame(calc.pandas(dataext['Molecule']))
data1=pd.concat([X,X2])
data1=data1._get_numeric_data()
X=data1.iloc[:265,:]
X2=data1.iloc[265:]
scaler = StandardScaler()
X = scaler.fit_transform(X)
X2 = scaler.transform(X2) 
default_cv = KFold(n_splits=10, shuffle=True, random_state=666)

 45%|████████████████████████████████████▏                                           | 120/265 [00:05<00:03, 40.13it/s]

C:\Users\11936\AppData\Roaming\Python\Python37\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|██████████████████████████████████████████████████████████████████████████████████| 66/66 [00:03<00:00, 17.04it/s]


In [2]:
import numpy as np
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
RF=RandomForestRegressor(max_depth=10, max_features='sqrt',min_samples_leaf= 3,min_samples_split=2,n_estimators=800,random_state=42)
SVR=SVR(C=0.0015, kernel='linear',max_iter= 900)
XGB=xgb.XGBRegressor(n_estimators=400,
    max_depth=9,
    learning_rate=1,
    gamma=4,
    subsample=0.90,
    colsample_bytree=0.85)
GBR=GradientBoostingRegressor(learning_rate=0.0177,max_depth=8, max_features='sqrt',min_samples_leaf= 5,min_samples_split=2,
                               n_estimators=200,random_state=42)


In [3]:
from mlxtend.regressor import StackingCVRegressor
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
## 两两集成
stack1 = StackingCVRegressor(regressors=[RF,SVR],
                            meta_regressor = lr,
                            use_features_in_secondary=False)  # #
stack2 = StackingCVRegressor(regressors=[RF,GBR],
                            meta_regressor = lr,
                            use_features_in_secondary=False )  # #
stack3 = StackingCVRegressor(regressors=[RF,XGB],
                            meta_regressor = lr,
                            use_features_in_secondary=False )  # #
stack4 = StackingCVRegressor(regressors=[XGB,SVR],
                            meta_regressor = lr,
                            use_features_in_secondary=False )  # #
stack5 = StackingCVRegressor(regressors=[XGB,GBR],
                            meta_regressor = lr,
                            use_features_in_secondary=False )  # #
stack6 = StackingCVRegressor(regressors=[GBR,SVR],
                            meta_regressor = lr,
                            use_features_in_secondary=False )  # #

In [4]:
## 三个集成
stack7 = StackingCVRegressor(regressors=[XGB,SVR,RF],
                            meta_regressor = lr,
                            use_features_in_secondary=False)  # #
stack8 = StackingCVRegressor(regressors=[XGB,SVR,GBR],
                            meta_regressor = lr,
                            use_features_in_secondary=False )  # #
stack9 = StackingCVRegressor(regressors=[XGB,GBR,RF],
                            meta_regressor = lr,
                            use_features_in_secondary=False )  # #
stack10 = StackingCVRegressor(regressors=[GBR,SVR,RF],
                            meta_regressor = lr,
                            use_features_in_secondary=False )  # #
#四个
stack11 = StackingCVRegressor(regressors=[XGB,SVR,RF,GBR],
                            meta_regressor = lr,
                            use_features_in_secondary=False )  # #

In [5]:
def fit_model(model):
    RANDOM_SEED = 42
    np.random.seed(RANDOM_SEED)
    model.fit(X,y) 
    fit = model.predict(X)
    pre = model.predict(X2)
    r2_tra = r2_score(y,fit)
    r2_ext = r2_score(y2, pre)
    rmse_tra = np.sqrt(mean_squared_error(y,fit))
    rmse_ext = np.sqrt(mean_squared_error(y2,pre))
    mae_tra = mean_absolute_error(y,fit)
    mae_ext = mean_absolute_error(y2,pre)
    return  r2_tra,r2_ext,rmse_tra, rmse_ext, mae_tra, mae_ext 

results = []
names = []
for model,label in zip([stack1,stack2,stack3,stack4,stack5,
                        stack6,stack7,stack8,stack9,stack10,stack11], 
                       ['1','2','3','4','5','6','7','8','9','10','11']): 
    names.append(label)
    score = fit_model(model)     #,xgb_re,'XGBoost',
    results.append(score)
results_df = pd.DataFrame(results,names)    


NameError: name 'cross_val_score' is not defined

In [6]:
from sklearn.model_selection import cross_val_score,KFold
def evaluate_model(model):
    cv = KFold(n_splits=10, shuffle=True, random_state=666)
    scores = cross_val_score(model, X, y, scoring='r2', cv=cv)
    #scores = np.sqrt(mean_squared_error(train_y.astype('int'),model.predict(train_x.values)))
    return scores
re = []
na = []
for model in [stack1,stack2,stack3,stack4,stack5,
                        stack6,stack7,stack8,stack9,stack10,stack11]: 
    score = evaluate_model(model)     #,xgb_re,'XGBoost',
    re.append(score)
    na.append(label)
    print("score: %0.3f (+/- %0.3f) "  % (score.mean(), score.std()))
plt.boxplot(re, labels=na, showmeans=True,showfliers= False,vert=True)


#放在一起比差异太小啦！
re_all = results + re
na_all = names + na


score: 0.738 (+/- 0.133) 
score: 0.765 (+/- 0.137) 
score: 0.754 (+/- 0.143) 
score: 0.734 (+/- 0.102) 
score: 0.770 (+/- 0.137) 
score: 0.758 (+/- 0.133) 
score: 0.744 (+/- 0.132) 
score: 0.770 (+/- 0.126) 
score: 0.768 (+/- 0.129) 
score: 0.758 (+/- 0.125) 
score: 0.766 (+/- 0.119) 


NameError: name 'plt' is not defined

In [11]:
results_df.to_csv("jicheng-BMF.csv")

In [19]:
pred1=pd.DataFrame(stack2.predict(X))
pred2=pd.DataFrame(stack2.predict(X2))
pred1.to_csv("stack2-BMF-train.csv")
pred2.to_csv("stack2-BMF-test.csv")

In [23]:
r2_tra = r2_score(y2,pred2)
r2_tra

0.790423681199539

In [24]:
r2_tra = r2_score(pred2,y2)
r2_tra

0.7626495140247337